In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp
from matplotlib import pyplot as plt
import seaborn as sns

In [ ]:
fashion_mnist = tf.keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

# preprocessing
train_images = train_images / 255.
test_images = test_images / 255.

# convert the numpy arrays to a tensorflow dataset
train_ds = tf.data.Dataset.from_tensor_slices((train_images, train_labels))

In [ ]:
class Autoencoder(tf.keras.models.Model):
    def __init__(self, latent_dim):
        super(Autoencoder, self).__init__()
        self.latent_dim = latent_dim
        self.prior = tfp.distributions.Independent(tfp.distributions.Normal(loc=tf.zeros(latent_dim), scale=1), reinterpreted_batch_ndims=1)

        latend_dim_params = tfp.layers.MultivariateNormalTriL.params_size(latent_dim)
        self.encoder = tf.keras.Sequential([
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(128),
            tf.keras.layers.Dense(latend_dim_params, activation=None),
            tfp.layers.MultivariateNormalTriL(
                latent_dim,
                activity_regularizer=tfp.layers.KLDivergenceRegularizer(self.prior)),
        ])

        self.decoder = tf.keras.Sequential([
            tf.keras.layers.InputLayer(input_shape=[latent_dim]),
            tf.keras.layers.Dense(784, activation='sigmoid'),
            tf.keras.layers.Reshape((28, 28))
        ])

    def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

latent_dim = 16 
autoencoder = Autoencoder(latent_dim)
autoencoder.compile(optimizer='adam', loss=tf.keras.losses.MeanSquaredError())

In [ ]:
history = autoencoder.fit(train_images, train_images,
                epochs=5,
                shuffle=True,
                validation_data=(test_images, test_images))

In [ ]:
encoded_imgs = autoencoder.encoder(test_images)
decoded_imgs = autoencoder.decoder(encoded_imgs).numpy()

In [ ]:
fig, ax = plt.subplots(figsize=(15, 4))
sns.heatmap(encoded_imgs[:20, :], ax=ax, square=True)
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.hist(encoded_imgs.sample().numpy().flat, bins=100)
plt.show()

In [ ]:
n = 10
fig = plt.figure(figsize=(20, 4))
for i in range(n):
    ax = fig.add_subplot(2, n, i + 1)
    ax.imshow(test_images[i])
    ax.set_title("original")
    
    ax = fig.add_subplot(2, n, i + 1 + n)
    ax.imshow(decoded_imgs[i])
    ax.set_title("reconstructed")
    
for ax in fig.get_axes():
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_aspect('equal')

plt.show()

In [ ]:
fig, axs = plt.subplots(1, 10, figsize=(15, 3))
for ax in axs.flat:
    noise = np.random.multivariate_normal(np.zeros(latent_dim), np.eye(latent_dim))
    noise = np.expand_dims(noise, axis=0)
    decoded_img = autoencoder.decoder(noise).numpy()[0]
    ax.imshow(decoded_img)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_aspect('equal')